In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline
from six.moves import cPickle
from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score

sys.path.append('..')
import helper
from deepomics import neuralnetwork as nn
from deepomics import utils, fit

In [2]:



def experiment_correpsondence(correspondence_path, experiments):
    rncmpt_names = []
    clip_names = []
    rbp_names = []
    cell_types = []
    with open(correspondence_path, 'rb') as f:
        for line in f:
            index = line.index('-')
            rncmpt_names.append(line[:index])
            experiment = line[index+1:].split()[0]
            index = experiment.index('_')
            rbp_names.append(experiment[:index])
            cell_types.append(experiment[index+1:])
            clip_names.append(experiment+'_200.h5')
            
    # get a dictionary of tuples for each rbp correspondence
    unique_rbps = np.unique(rbp_names)
    match = {}
    for j, rbp_name in enumerate(unique_rbps):
        match[rbp_name] = []
    for j, rbp_name in enumerate(rbp_names):
        rbp_index = np.where(experiments==rncmpt_names[j])[0][0]
        match[rbp_name].append((rbp_index, rncmpt_names[j], clip_names[j]))
    return match

def binding_affinity_scores(sess, nntrainer, clip_train):

    X_train = clip_train['inputs']
    y_train = clip_train['targets']

    index = np.where(y_train[:,0]==1)[0]
    X_pos = X_train[index]
    y_pos = y_train[index]
    index = np.where(y_train[:,0]==0)[0]
    X_neg = X_train[index]
    y_neg = y_train[index]

    num_split = X_pos.shape[1] - 41

    pos_score = []
    pos_max_score = []
    pos_mean_score = []
    for X in X_pos:
        X_split = []
        for i in range(num_split):
            X_split.append([X[i:i+41,:,:]])
        X_split = np.vstack(X_split)

        affinity = nntrainer.get_activations(sess, {'inputs': X_split})
        affinity = affinity[::-1,0]
        pos_max_score.append(affinity[0])
        pos_mean_score.append(np.mean(affinity))
        pos_score.append(np.mean(affinity[:25]))
        
    pos_score = np.array(pos_score)
    pos_max_score = np.array(pos_max_score)
    pos_mean_score = np.array(pos_mean_score)

    neg_score = []
    neg_max_score = []
    neg_mean_score = []
    for X in X_neg:
        X_split = []
        for i in range(num_split):
            X_split.append([X[i:i+41,:,:]])
        X_split = np.vstack(X_split)

        affinity = nntrainer.get_activations(sess, {'inputs': X_split})
        affinity = affinity[::-1,0]
        neg_max_score.append(affinity[0])
        neg_mean_score.append(np.mean(affinity))
        neg_score.append(np.mean(affinity[:25]))
    neg_score = np.array(neg_score)
    neg_mean_score = np.array(neg_mean_score)
    neg_max_score = np.array(neg_max_score)

    y_true = np.vstack([np.ones((len(pos_score),1)), np.zeros((len(neg_score),1))])
    y_score = np.vstack([np.expand_dims(pos_score,axis=1), np.expand_dims(neg_score, axis=1)])
    y_mean_score = np.vstack([np.expand_dims(pos_mean_score,axis=1), np.expand_dims(neg_mean_score, axis=1)])
    y_max_score = np.vstack([np.expand_dims(pos_max_score,axis=1), np.expand_dims(neg_max_score, axis=1)])

    return y_true, y_score, y_mean_score, y_max_score


In [3]:

# get list of encode-eclip experiments
data_path = '../../data/RNAcompete_2013/rnacompete2013.h5'
experiments = helper.get_experiments_hdf5(data_path)

# get corresponding clip-experiments for rnacompete experiments
correspondence_path = 'correspondences_eCLIP_RNACompete.txt'
match = experiment_correpsondence(correspondence_path, experiments)
    

In [4]:
match

{'FMR1': [(60, 'RNCMPT00015', 'FMR1_K562_200.h5'),
  (71, 'RNCMPT00016', 'FMR1_K562_200.h5')],
 'FXR1': [(62, 'RNCMPT00161', 'FXR1_K562_200.h5')],
 'FXR2': [(102, 'RNCMPT00020', 'FXR2_K562_200.h5')],
 'HNRNPA1': [(117, 'RNCMPT00022', 'HNRNPA1_HepG2_200.h5'),
  (117, 'RNCMPT00022', 'HNRNPA1_K562_200.h5')],
 'HNRNPC': [(143, 'RNCMPT00025', 'HNRNPC_HepG2_200.h5')],
 'HNRNPK': [(150, 'RNCMPT00026', 'HNRNPK_HepG2_200.h5'),
  (150, 'RNCMPT00026', 'HNRNPK_K562_200.h5')],
 'IGF2BP2': [(173, 'RNCMPT00033', 'IGF2BP2_K562_200.h5')],
 'IGF2BP3': [(74, 'RNCMPT00172', 'IGF2BP3_HepG2_200.h5')],
 'KHDRBS1': [(70, 'RNCMPT00169', 'KHDRBS1_K562_200.h5'),
  (204, 'RNCMPT00062', 'KHDRBS1_K562_200.h5')],
 'PCBP2': [(185, 'RNCMPT00044', 'PCBP2_HepG2_200.h5')],
 'PTBP1': [(148, 'RNCMPT00268', 'PTBP1_HepG2_200.h5'),
  (149, 'RNCMPT00269', 'PTBP1_HepG2_200.h5'),
  (148, 'RNCMPT00268', 'PTBP1_K562_200.h5'),
  (149, 'RNCMPT00269', 'PTBP1_K562_200.h5')],
 'QKI': [(188, 'RNCMPT00047', 'QKI_HepG2_200.h5'),
  (188, '

In [17]:
model = 'affinity_conv_net'
# model = 'affinity_residual_net'
rbp = 'FMR1'
rbp_index = 60
rncmpt_name = 'RNCMPT00015'
clip_name = 'FMR1_K562_200.h5'

normalize_method = 'log_norm' 
ss_type = 'seq'

# load rbp dataset
train, valid, test = helper.load_dataset_hdf5(data_path, ss_type=ss_type, rbp_index=rbp_index)

# process rbp dataset
train, valid, test = helper.process_data(train, valid, test, method=normalize_method)

# get shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = train['targets'].shape

# load model
genome_model = helper.import_model(model)
model_layers, optimization = genome_model.model(input_shape, output_shape)

# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization, use_scope=False)

results_path = helper.make_directory('../../results', 'RNAcompete_2013')
file_path = os.path.join(results_path, normalize_method+'_'+ss_type, model, rncmpt_name)
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

# initialize session
sess = utils.initialize_session(nnmodel.placeholders)

# load best model
nntrainer.set_best_parameters(sess)
loss, mean, std = nntrainer.test_model(sess, test)

dataset_path = '/media/peter/storage/encode_eclip/eclip_datasets'
dataset_file_path = os.path.join(dataset_path, clip_name)
clip_train, clip_valid, clip_test = helper.load_dataset_hdf5(dataset_file_path, ss_type=ss_type)


y_true, y_score, y_mean_score, y_max_score = binding_affinity_scores(sess, nntrainer, clip_train)

from sklearn.metrics import roc_curve, auc, precision_recall_curve, accuracy_score, roc_auc_score
fpr, tpr, thresholds = roc_curve(y_true, y_max_score)
roc_score = auc(fpr, tpr)
print(roc_score)

fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_score = auc(fpr, tpr)
print(roc_score)

fpr, tpr, thresholds = roc_curve(y_true, y_mean_score)
roc_score = auc(fpr, tpr)
print(roc_score)



loading model from:  ../../results/RNAcompete_2013/log_norm_seq/affinity_conv_net/RNCMPT00015_best.ckpt
  test  loss:		0.73678
  test  Pearson's R:	0.51453+/-0.00000
  test  rsquare:	0.26459+/-0.00000
  test  slope:		1.06201+/-0.00000
0.446273126939
0.483142449422
0.427068274714


In [5]:
models = ['affinity_residual_net', 'affinity_conv_net']
normalize_method = 'log_norm' 
ss_type = 'seq'
best_path = '../../results/RNAcompete_2013/'+normalize_method+'_'+ss_type

# get list of rnacompete experiments
data_path = '../../data/RNAcompete_2013/rnacompete2013.h5'
experiments = helper.get_experiments_hdf5(data_path)

# get corresponding clip-experiments for rnacompete experiments
correspondence_path = 'correspondences_eCLIP_RNACompete.txt'
match = experiment_correpsondence(correspondence_path, experiments)

# directory for encode-clip experiments
dataset_path = '/media/peter/storage/encode_eclip/eclip_datasets'


results = []
for key in match.keys():
    for experiments in match[key]:
        rbp_index = experiments[0]
        rcmpt_name = experiments[1]
        clip_name = experiments[2]

        try:
            # load clip dataset
            dataset_file_path = os.path.join(dataset_path, clip_name)
            clip_train, clip_valid, clip_test = helper.load_dataset_hdf5(dataset_file_path, ss_type=ss_type)

            inputs = np.vstack([clip_train['inputs'], clip_valid['inputs'], clip_test['inputs']])
            targets = np.vstack([clip_train['targets'], clip_valid['targets'], clip_test['targets']])
            clip_train = {'inputs': inputs, 'targets': targets}

            # load rbp dataset
            train, valid, test = helper.load_dataset_hdf5(data_path, ss_type=ss_type, rbp_index=rbp_index)

            # process rbp dataset
            train, valid, test = helper.process_data(train, valid, test, method=normalize_method)

            # get shapes
            input_shape = list(train['inputs'].shape)
            input_shape[0] = None
            output_shape = train['targets'].shape

            for model in models:

                # load model
                genome_model = helper.import_model(model)
                model_layers, optimization = genome_model.model(input_shape, output_shape)

                # build neural network class
                nnmodel = nn.NeuralNet(seed=247)
                nnmodel.build_layers(model_layers, optimization, use_scope=False)

                file_path = os.path.join(best_path, model, rcmpt_name)
                nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

                # initialize session
                sess = utils.initialize_session(nnmodel.placeholders)

                # load best model
                nntrainer.set_best_parameters(sess, verbose=0)

                loss, mean, std = nntrainer.test_model(sess, test, verbose=0)

                # get scores for each protein
                y_true, y_score, y_mean_score, y_max_score = binding_affinity_scores(sess, nntrainer, clip_train)

                fpr, tpr, thresholds = roc_curve(y_true, y_max_score)
                roc_score = auc(fpr, tpr)
                fpr, tpr, thresholds = roc_curve(y_true, y_score)
                roc_score2 = auc(fpr, tpr)
                fpr, tpr, thresholds = roc_curve(y_true, y_mean_score)
                roc_score3 = auc(fpr, tpr)

                print("%s\t%s\t%s\t%0.4f\t%0.4f\t%0.4f\t%0.4f"%(rcmpt_name, clip_name, model, mean[0], roc_score, roc_score2, roc_score3))
                results.append([rcmpt_name, clip_name, model, mean[0], roc_score, roc_score2, roc_score3])
        except:
            print(clip_name + ' not found')

RNCMPT00047	QKI_HepG2_200.h5	affinity_residual_net	0.0301	0.6601	0.7011	0.8344
RNCMPT00047	QKI_HepG2_200.h5	affinity_conv_net	0.0326	0.6233	0.6611	0.8444
RNCMPT00047	QKI_K562_200.h5	affinity_residual_net	0.0301	0.6436	0.6908	0.8635
RNCMPT00047	QKI_K562_200.h5	affinity_conv_net	0.0326	0.6234	0.6527	0.8847
RNCMPT00033	IGF2BP2_K562_200.h5	affinity_residual_net	0.8000	0.4678	0.4750	0.4295
RNCMPT00033	IGF2BP2_K562_200.h5	affinity_conv_net	0.7550	0.4597	0.4662	0.4077
SRSF1_HepG2_200.h5 not found
SRSF1_HepG2_200.h5 not found
SRSF1_HepG2_200.h5 not found
SRSF1_HepG2_200.h5 not found
SRSF1_HepG2_200.h5 not found
SRSF1_HepG2_200.h5 not found
RNCMPT00106	SRSF1_K562_200.h5	affinity_residual_net	0.7472	0.6375	0.6659	0.8446
RNCMPT00106	SRSF1_K562_200.h5	affinity_conv_net	0.7909	0.6413	0.6625	0.8584
RNCMPT00107	SRSF1_K562_200.h5	affinity_residual_net	0.8664	0.6501	0.6687	0.8619
RNCMPT00107	SRSF1_K562_200.h5	affinity_conv_net	0.8427	0.6473	0.6702	0.8596
RNCMPT00108	SRSF1_K562_200.h5	affinity_residual_

In [16]:
match

{'FMR1': [(60, 'RNCMPT00015', 'FMR1_K562_200.h5'),
  (71, 'RNCMPT00016', 'FMR1_K562_200.h5')],
 'FXR1': [(62, 'RNCMPT00161', 'FXR1_K562_200.h5')],
 'FXR2': [(102, 'RNCMPT00020', 'FXR2_K562_200.h5')],
 'HNRNPA1': [(117, 'RNCMPT00022', 'HNRNPA1_HepG2_200.h5'),
  (117, 'RNCMPT00022', 'HNRNPA1_K562_200.h5')],
 'HNRNPC': [(143, 'RNCMPT00025', 'HNRNPC_HepG2_200.h5')],
 'HNRNPK': [(150, 'RNCMPT00026', 'HNRNPK_HepG2_200.h5'),
  (150, 'RNCMPT00026', 'HNRNPK_K562_200.h5')],
 'IGF2BP2': [(173, 'RNCMPT00033', 'IGF2BP2_K562_200.h5')],
 'IGF2BP3': [(74, 'RNCMPT00172', 'IGF2BP3_HepG2_200.h5')],
 'KHDRBS1': [(70, 'RNCMPT00169', 'KHDRBS1_K562_200.h5'),
  (204, 'RNCMPT00062', 'KHDRBS1_K562_200.h5')],
 'PCBP2': [(185, 'RNCMPT00044', 'PCBP2_HepG2_200.h5')],
 'PTBP1': [(148, 'RNCMPT00268', 'PTBP1_HepG2_200.h5'),
  (149, 'RNCMPT00269', 'PTBP1_HepG2_200.h5'),
  (148, 'RNCMPT00268', 'PTBP1_K562_200.h5'),
  (149, 'RNCMPT00269', 'PTBP1_K562_200.h5')],
 'QKI': [(188, 'RNCMPT00047', 'QKI_HepG2_200.h5'),
  (188, '

In [32]:
rbp_index = 220
rcmpt_name = 'RNCMPT00077'
model = 'affinity_residual_net'
normalize_method = 'log_norm' 
ss_type = 'seq'
best_path = '../../results/RNAcompete_2013/'+normalize_method+'_'+ss_type

# get list of rnacompete experiments
data_path = '../../data/RNAcompete_2013/rnacompete2013.h5'
experiments = helper.get_experiments_hdf5(data_path)

# load rbp dataset
train, valid, test = helper.load_dataset_hdf5(data_path, ss_type=ss_type, rbp_index=rbp_index)

# process rbp dataset
train, valid, test = helper.process_data(train, valid, test, method=normalize_method)

# get shapes
input_shape = list(train['inputs'].shape)
input_shape[0] = None
output_shape = train['targets'].shape

# load model
genome_model = helper.import_model(model)
model_layers, optimization = genome_model.model(input_shape, output_shape)

# build neural network class
nnmodel = nn.NeuralNet(seed=247)
nnmodel.build_layers(model_layers, optimization, use_scope=False)

# compile neural trainer
file_path = os.path.join(best_path, model, rcmpt_name)
nntrainer = nn.NeuralTrainer(nnmodel, save='best', file_path=file_path)

# initialize session
sess = utils.initialize_session(nnmodel.placeholders)

# load best model
nntrainer.set_best_parameters(sess)

# test model on validation set
loss, mean_vals, std_vals = nntrainer.test_model(sess, test, batch_size=128, name='test', verbose=1)


loading model from:  ../../results/RNAcompete_2013/log_norm_seq/affinity_residual_net/RNCMPT00077_best.ckpt
  test  loss:		0.30755
  test  Pearson's R:	0.83684+/-0.00000
  test  rsquare:	0.69963+/-0.00000
  test  slope:		0.99765+/-0.00000


In [34]:
clip_name = 'TIA1_K562_200.h5'

# directory for encode-clip experiments
dataset_path = '/media/peter/storage/encode_eclip/eclip_datasets'
dataset_file_path = os.path.join(dataset_path, clip_name)
clip_train, clip_valid, clip_test = helper.load_dataset_hdf5(dataset_file_path, ss_type=ss_type)

inputs = np.vstack([clip_train['inputs'], clip_valid['inputs'], clip_test['inputs']])
targets = np.vstack([clip_train['targets'], clip_valid['targets'], clip_test['targets']])
clip_train = {'inputs': inputs, 'targets': targets}

# get scores for each protein
y_true, y_score, y_mean_score, y_max_score = binding_affinity_scores(sess, nntrainer, clip_train)

fpr, tpr, thresholds = roc_curve(y_true, y_max_score)
roc_score = auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_score2 = auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_true, y_mean_score)
roc_score3 = auc(fpr, tpr)
print("%s\t%s\t%s\t%0.4f\t%0.4f\t%0.4f\t%0.4f"%(rcmpt_name, clip_name, model, mean_vals[0], roc_score, roc_score2, roc_score3))


RNCMPT00077	TIA1_K562_200.h5	affinity_residual_net	0.8368	0.5867	0.5981	0.6762


In [29]:
clip_name = 'TIA1_K562_200.h5'

# directory for encode-clip experiments
dataset_path = '/media/peter/storage/encode_eclip/eclip_datasets'
dataset_file_path = os.path.join(dataset_path, clip_name)
clip_train, clip_valid, clip_test = helper.load_dataset_hdf5(dataset_file_path, ss_type=ss_type)

inputs = np.vstack([clip_train['inputs'], clip_valid['inputs'], clip_test['inputs']])
targets = np.vstack([clip_train['targets'], clip_valid['targets'], clip_test['targets']])
clip_train = {'inputs': inputs, 'targets': targets}

# get scores for each protein
y_true, y_score, y_mean_score, y_max_score = binding_affinity_scores(sess, nntrainer, clip_train)

fpr, tpr, thresholds = roc_curve(y_true, y_max_score)
roc_score = auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_score2 = auc(fpr, tpr)
fpr, tpr, thresholds = roc_curve(y_true, y_mean_score)
roc_score3 = auc(fpr, tpr)
print("%s\t%s\t%s\t%0.4f\t%0.4f\t%0.4f\t%0.4f"%(rcmpt_name, clip_name, model, mean_vals[0], roc_score, roc_score2, roc_score3))


RNCMPT00165	TIA1_K562_200.h5	affinity_conv_net	0.7739	0.5906	0.6024	0.7186


In [27]:
clip_name = 'PTBP1_HepG2_200.h5'

# directory for encode-clip experiments
dataset_path = '/media/peter/storage/encode_eclip/eclip_datasets'
dataset_file_path = os.path.join(dataset_path, clip_name)
#clip_train, clip_valid, clip_test = helper.load_dataset_hdf5(dataset_file_path, ss_type=ss_type)
dataset = h5py.File(dataset_file_path)
X_train = np.array(dataset['X_train']).astype(np.float32)        
y_train = np.array(dataset['Y_train']).astype(np.float32)
X_valid = np.array(dataset['X_valid']).astype(np.float32)       
y_valid = np.array(dataset['Y_valid']).astype(np.float32)
X_test = np.array(dataset['X_test']).astype(np.float32)
y_test = np.array(dataset['Y_test']).astype(np.float32)

# add another dimension to make a 4d tensor
X_train = np.expand_dims(X_train, axis=3).transpose([0, 2, 3, 1])
X_test = np.expand_dims(X_test, axis=3).transpose([0, 2, 3, 1])
X_valid = np.expand_dims(X_valid, axis=3).transpose([0, 2, 3, 1])

# dictionary for each dataset
clip_train = {'inputs': X_train[:,:,:,:4], 'targets': y_train}
clip_valid = {'inputs': X_valid[:,:,:,:4], 'targets': y_valid}
clip_test = {'inputs': X_test[:,:,:,:4], 'targets': y_test}


inputs = np.vstack([clip_train['inputs'], clip_valid['inputs'], clip_test['inputs']])
targets = np.vstack([clip_train['targets'], clip_valid['targets'], clip_test['targets']])
clip_train = {'inputs': inputs, 'targets': targets}
X_train = clip_train['inputs']
y_train = clip_train['targets']

index = np.where(y_train[:,0]==1)[0]
X_pos = X_train[index]
y_pos = y_train[index]
index = np.where(y_train[:,0]==0)[0]
X_neg = X_train[index]
y_neg = y_train[index]

pos_score = []
pos_max_score = []
pos_mean_score = []
for X in X_pos:
    X_split = []
    for i in range(num_split):
        X_split.append([X[i:i+41,:,:]])
    X_split = np.vstack(X_split)

    affinity = nntrainer.get_activations(sess, {'inputs': X_split})
    affinity = affinity[::-1,0]
    pos_max_score.append(affinity[0])
    pos_mean_score.append(np.mean(affinity))
    pos_score.append(np.mean(affinity[:25]))

pos_score = np.array(pos_score)
pos_max_score = np.array(pos_max_score)
pos_mean_score = np.array(pos_mean_score)

neg_score = []
neg_max_score = []
neg_mean_score = []
for X in X_neg:
    X_split = []
    for i in range(num_split):
        X_split.append([X[i:i+41,:,:]])
    X_split = np.vstack(X_split)

    affinity = nntrainer.get_activations(sess, {'inputs': X_split})
    affinity = affinity[::-1,0]
    neg_max_score.append(affinity[0])
    neg_mean_score.append(np.mean(affinity))
    neg_score.append(np.mean(affinity[:25]))
neg_score = np.array(neg_score)
neg_mean_score = np.array(neg_mean_score)
neg_max_score = np.array(neg_max_score)

y_true = np.vstack([np.ones((len(pos_score),1)), np.zeros((len(neg_score),1))])
y_score = np.vstack([np.expand_dims(pos_score,axis=1), np.expand_dims(neg_score, axis=1)])
y_mean_score = np.vstack([np.expand_dims(pos_mean_score,axis=1), np.expand_dims(neg_mean_score, axis=1)])
y_max_score = np.vstack([np.expand_dims(pos_max_score,axis=1), np.expand_dims(neg_max_score, axis=1)])

fpr, tpr, thresholds = roc_curve(y_true, y_max_score)
roc_score = auc(fpr, tpr)
print(roc_score)

fpr, tpr, thresholds = roc_curve(y_true, y_score)
roc_score = auc(fpr, tpr)
print(roc_score)

fpr, tpr, thresholds = roc_curve(y_true, y_mean_score)
roc_score = auc(fpr, tpr)
print(roc_score)


NameError: name 'num_split' is not defined